# Visualizing Vector Data - Cities/Towns

### Note: In this example, I will use Oregon major cities shapefile. Link to the resource: [Oregon Spatial Data Library](https://spatialdata.oregonexplorer.info/geoportal/search)

In [1]:
import geopandas as gpd
from ipyleaflet import Map, CircleMarker, LayersControl, GeoData, basemaps
from shapely import wkt
from shapely.ops import transform
from pyproj import CRS, Transformer

In [2]:
# using codes below to stop displaying "warnings"
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [3]:
# read file with gpd
cities = gpd.read_file("D:\cities.shp") # your file path
cities.head()

,AREA,PERIMETER,CITIES_,CITIES_ID,CITY,FLAG,geometry
0,0.0,0.0,1,1658,MULINO,0,POINT (776899.812 1272019.125)
1,0.0,0.0,2,1368,HAMMOND,0,POINT (439320.844 1638725.000)
2,0.0,0.0,3,1366,FORT STEVENS,0,POINT (435108.000 1641590.750)
3,0.0,0.0,4,1382,GLIFTON,0,POINT (560721.000 1638699.875)
4,0.0,0.0,5,1384,BRADWOOD,0,POINT (568325.812 1632800.000)


In [4]:
# Function to convert geometries to 2D
def to_2d(geometry):
    if geometry.is_empty:
        return geometry
    if '3D' in geometry.type:
        # Convert MultiPolygon and Polygon geometries to 2D
        return wkt.loads(wkt.dumps(geometry, output_dimension=2))
    return geometry

# Ensure all geometries are valid and convert to 2D
cities = cities[cities.geometry.notnull() & cities.is_valid]
cities['geometry'] = cities['geometry'].apply(to_2d)

# Define source and target CRS using pyproj
source_crs = CRS(cities.crs)
target_crs = CRS('epsg:4326') # you can set your desired crs here
project = Transformer.from_crs(source_crs, target_crs, always_xy=True).transform

# Apply the transformation
cities['geometry'] = cities['geometry'].apply(lambda geom: transform(project, geom))

# Create an ipyleaflet map
m = Map(center=(43.80, -120.55), zoom=6, basemap=basemaps.OpenStreetMap.Mapnik) # set center depends on your project, zoom as well

# Loop through each city and create a circle marker
for idx, row in cities.iterrows():
    circle_marker = CircleMarker()
    circle_marker.location = (row.geometry.y, row.geometry.x)
    circle_marker.radius = 1  # Adjust the size as needed
    circle_marker.color = "black"
    circle_marker.weight = 1  # Adjust the thickness of the circle's outline
    m.add_layer(circle_marker)

# Add layer control
m.add_control(LayersControl())

# Display the map
m


Map(center=[43.8, -120.55], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…